# 1ª Avaliação de Programação Estatística com Python - 10/12/2019

- Nome: Matteus Silvestre Maciel Das Neves Carvalho
- Matrícula: 20180005860

**Questão 1 (1,0 ponto)** Implemente uma função que gere $C$ matrizes $\mathbf{X}_c^{n_c \times p}$, $c = 1, \ldots, C$, aleatoriamente, onde $n_c$ é a quantidade de linhas da matriz $\mathbf{X}_c$. As $p$ colunas de cada matriz $\mathbf{X}_c$ devem ser sorteadas de distribuições normais com médias e variâncias diferentes. A função deve então concatenar todas as matrizes geradas e retornar a matriz $\mathbf{X}^{n \times p}$, onde $n = \sum_{c=1}^C n_c$.    
__Dica 1__: A função pode receber como parâmetros uma lista com a quantidade de linhas de cada matriz $\mathbf{X}_c$, uma lista com as médias das normais usadas para gerar as colunas de cada matriz e outra lista com as variâncias correspondentes.    
__Dica 2__: considere usar a função *numpy.random.multivariate_normal*.

In [1]:
import numpy as np

In [2]:
def multiv_normal_matrix(n, means, covs):
    # A primeira matriz
    X = np.random.multivariate_normal(means[0], covs[0], size = n[0])
    # Caso C > 1, gerar mais matrizes e concatená-las
    # sucessivamente com a primeira:
    for i in range(1,len(n)):
        X_c = np.random.multivariate_normal(means[i], covs[i], size = n[i])
        X = np.vstack((X, X_c))
    return X

Exemplo:

In [3]:
# Fixando a semente para garantir a replicabilidade
# dos resultados
np.random.seed(7122019)

# Quantidade nc de linhas das matrizes, médias das v.a.s
# normais e matrizes de covariância:
nc = [3,4,2]
means = [[0, 5, 10], [-5, 0, 5], [10, -5, -10]]
covs = np.array([[[1, 0, 0], [0, 4, 0], [0, 0, 0.5]],
                [[9,0,0], [0, 1, 0], [0, 0, 9]],
                [[0.5, 0, 0], [0, 9, 0], [0, 0, 0.5]]])

X = multiv_normal_matrix(nc, means, covs)
print(X)

[[  0.18544677   3.44532934   9.8875874 ]
 [ -0.88329932   4.40609639  10.09076111]
 [ -1.30690015   6.34527367   8.98891742]
 [ -3.26805774  -0.11190506   3.99637801]
 [ -7.03039126  -0.31730128   4.19356572]
 [ -2.42656416   0.14895415   4.6408355 ]
 [ -3.52520519   0.82861773   5.17268383]
 [  9.39943971  -3.87473391 -10.83395248]
 [ 10.52442769  -3.31386615  -8.67614233]]


**Questão 2 (1,0 ponto)** Faça uma função que recebe uma matriz $\mathbf{X}^{n \times p}$ e um inteiro $k$ e retorna uma matriz $W^{k \times p}$, cujas linhas foram selecionadas aleatoriamente e sem reposição de $\mathbf{X}^{n \times p}$.

In [4]:
def rsample(X, k):
    # Gera-se uma matriz Y, cópia de X
    Y = X.copy()
    # Reordena-se as linhas de X
    np.random.shuffle(Y)
    # Retorna-se as k primeiras linhas
    W = Y[0:k]
    return W

Exemplo:

In [5]:
np.random.seed(7122019)
X = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12]])
W = rsample(X,2)

print(X)
print()
print(W)

[[ 1  2  3]
 [ 4  5  6]
 [ 7  8  9]
 [10 11 12]]

[[ 7  8  9]
 [10 11 12]]


**Questão 3 (1,0 ponto)** Faça uma função que recebe duas matrizes $\mathbf{X}^{n \times p}$ e $\mathbf{W}^{k \times p}$ e retorna a matriz de distâncias Euclidianas $\mathbf{D}^{n \times k}$ entre as linhas de $\mathbf{X}$ e as de $\mathbf{W}$.

In [6]:
def dist_matrix(X, W):
    # Obtem-se as dimensões de X e W
    n = np.shape(X)[0]
    k = np.shape(W)[0]
    p = np.shape(X)[1]
    # Gera-se uma matriz D de zeros
    D = np.zeros((n, k))
    
    for l_x in range(n):
        for l_w in range(k):
            for coluna in range(p):
                D[l_x, l_w] += (X[l_x, coluna] - W[l_w, coluna])**2
    return D**(1/2)

Exemplo:

In [7]:
np.random.seed(7122019)
X = np.random.randint(-5,6,10).reshape(5,2) # Matriz 5 x 2 de pontos
W = np.array([[0,0], [3,0], [0,3]]) # Matriz 3 x 2 de pontos
D = dist_matrix(X,W) # Matriz 5 x 3 com as distâncias associadas a
                     # cada ponto de X a um de W
print(X)
print()
print(W)
print()
print(D)

[[ 4  3]
 [ 4  1]
 [-2  1]
 [ 5  1]
 [ 2  4]]

[[0 0]
 [3 0]
 [0 3]]

[[5.         3.16227766 4.        ]
 [4.12310563 1.41421356 4.47213595]
 [2.23606798 5.09901951 2.82842712]
 [5.09901951 2.23606798 5.38516481]
 [4.47213595 4.12310563 2.23606798]]


**Questão 4 (1,0 ponto)** Qual função de NumPy pode ser usada para obter para cada linha da matriz $\mathbf{D}$, retornada pela função da questão acima, a coluna que representa a menor distância da linha correspondente em $\mathbf{X}$ para as linhas de $\mathbf{W}$? Use a função para retornar essa informação.

In [8]:
# A função de Numpy apply_along_axis() permite que uma função seja
# aplicada em um array, percorrendo-o em relação a um de seus eixos.
# Já a função argmin(), também de Numpy, retorna a posição do menor
# elemento de um array.

# A questão pede a posição do valor mínimo (função) dentre as colunas (eixo 1) de
# cada linha da matriz D (array). Portanto, esse resultado é obtido por:

np.apply_along_axis(np.argmin, 1, D)

array([1, 1, 0, 1, 2], dtype=int64)

**Questão 5 (2,0 pontos)** Faça uma função que atualiza as linhas de $\mathbf{W}$, fazendo com que cada linha assuma o valor médio das linhas de $\mathbf{X}$ para as quais a mesma foi a mais próxima. A função deve retornar a matriz $\mathbf{W}$ modificada. Use a função de NumPy mencionada na Questão 4.

In [9]:
def avg_XW(X, W):
    
    n = np.shape(X)[0] # Número de linhas de X
    k = np.shape(W)[0] # Número de linhas de W
    D = dist_matrix(X, W) # Matriz com as distâncias entre as linhas de X e W
    
    # Array com as linhas de W mais próximas a cada linha de X
    W_prox = np.apply_along_axis(np.argmin, 1, D)
    
    # Matriz W modificada
    W_mod = np.zeros(np.shape(W))
    
    # Média das linhas de X mais próximas a cada elemento de W:
    for i in range(k):
        mins = 0
        for j in range(n):           
            if W_prox[j] == i:
                W_mod[i] += X[j]
                mins += 1
        # Para o caso onde alguma linha de W não é a mais próxima para
        # nenhuma linha de X, se deve previnir a divisão por zero.
        if mins == 0:
            mins = 1
        W_mod[i] = (W_mod[i] / mins)

    return W_mod

Exemplo:

In [10]:
# Usando as mesmas matrizes da questão anterior:
np.random.seed(7122019)
X = np.random.randint(-5,6,10).reshape(5,2)
W = np.array([[0,0], [3,0], [0,3]])
D = dist_matrix(X,W)
print(X)
print()
print(W)
print()
print(D)
print()
W_mod = avg_XW(X,W)
print()
print(W_mod)

[[ 4  3]
 [ 4  1]
 [-2  1]
 [ 5  1]
 [ 2  4]]

[[0 0]
 [3 0]
 [0 3]]

[[5.         3.16227766 4.        ]
 [4.12310563 1.41421356 4.47213595]
 [2.23606798 5.09901951 2.82842712]
 [5.09901951 2.23606798 5.38516481]
 [4.47213595 4.12310563 2.23606798]]


[[-2.          1.        ]
 [ 4.33333333  1.66666667]
 [ 2.          4.        ]]


**Questão 6 (1,5 pontos)** Implemente uma classe, chamada *KMeans*, cujo construtor define os seguintes atributos: $k$ e $t_{max}$. A classe deve conter um método *fit*, que recebe como parâmetro uma matriz $\mathbf{X}$. O método *fit* irá então seguir os seguintes passos:

1. Use a função implementada na Questão 2 para obter a matriz $\mathbf{W}$ (note que o segundo parâmetro da função receberá como argumento um dos atributos definidos no construtor);
2. Repita as operações abaixo $t_{max}$ vezes (note que isso é um atributo):    
    A. Use a função da Questão 3 para calcular as distâncias entre $\mathbf{X}$ e $\mathbf{W}$    
    B. Use a função da Questão 5 para atualizar as linhas de $\mathbf{W}$

Ao final do método fit, a matriz $\mathbf{W}$ resultante deve ser guardada como um atributo.

In [11]:
class KMeans:
    def __init__(self, k, tmax):
        self.k = k
        self.tmax = tmax
    def fit(self, X):
        W = rsample(X,self.k)
        for i in range(self.tmax):           
            W = avg_XW(X,W)
        self.W = W

Exemplo:

In [12]:
np.random.seed(7122019)
KM = KMeans(3, 10)

X = 10*np.random.random(30).reshape((10,3))
print(X)
print()

KM.fit(X)
print(KM.W)

[[5.98905728 0.85417875 0.71521686]
 [2.70610853 5.5891685  0.94554379]
 [2.4095396  6.33326107 4.33563529]
 [2.16038629 1.07980959 8.18982537]
 [4.12732617 9.50209211 1.79611561]
 [9.12236753 0.93009858 3.38350959]
 [3.28493483 0.45451419 5.70774676]
 [9.07584218 9.67105005 5.54693358]
 [9.94364387 3.37678032 6.67667902]
 [8.70399316 3.27684632 2.6071873 ]]

[[7.71560407 7.51664083 4.67324274]
 [2.72266056 0.76716189 6.94878606]
 [5.78621322 3.39671064 2.39741857]]


**Questão 7 (1,0 ponto)** Adicione o método *predict* à classe *KMeans*. O método irá receber uma nova matriz $\mathbf{X}$ e irá usar a função de NumPy mencionada na Questão 4 para retornar os índices das linhas de $\mathbf{W}$ mais próximas às linhas de $\mathbf{X}$.

In [13]:
class KMeans2(KMeans):
    def predict(self, X):
        D = dist_matrix(X, self.W)
        mais_proximos = np.apply_along_axis(np.argmin, 1, D)
        return mais_proximos

Exemplo:

In [14]:
# Treinando o modelo com os dados da Questão 6
np.random.seed(7122019)
KM = KMeans2(3, 10)

X_train = 10*np.random.random(30).reshape((10,3))
print(X_train)
print()
KM.fit(X_train)
print(KM.W)
print()

# E testando com uma nova matriz X:
np.random.seed(6122019)
X_test = 10*np.random.random(30).reshape((10,3))
print(X_test)
print()
print(KM.predict(X_test))

[[5.98905728 0.85417875 0.71521686]
 [2.70610853 5.5891685  0.94554379]
 [2.4095396  6.33326107 4.33563529]
 [2.16038629 1.07980959 8.18982537]
 [4.12732617 9.50209211 1.79611561]
 [9.12236753 0.93009858 3.38350959]
 [3.28493483 0.45451419 5.70774676]
 [9.07584218 9.67105005 5.54693358]
 [9.94364387 3.37678032 6.67667902]
 [8.70399316 3.27684632 2.6071873 ]]

[[7.71560407 7.51664083 4.67324274]
 [2.72266056 0.76716189 6.94878606]
 [5.78621322 3.39671064 2.39741857]]

[[1.75700393 1.59491685 9.6645511 ]
 [9.255594   7.87658576 9.70541694]
 [6.59413172 7.12764595 0.43241671]
 [1.35201539 8.4707409  6.34732478]
 [4.5306859  3.47221748 5.1326979 ]
 [5.90729181 3.4516044  9.48961166]
 [7.31914307 2.9305075  4.32998883]
 [5.20007376 4.43314179 1.7743091 ]
 [1.57399981 0.45514553 4.40701687]
 [7.23453043 8.83256176 9.52587947]]

[1 0 2 0 2 1 2 2 1 0]


**Questão 8 (1,5 ponto)** Adicione o método *score* à classe *KMeans*. O método irá receber uma nova matriz $\mathbf{X}$ e irá usar a função da Questão 3 para calcular as distâncias para a matriz $\mathbf{W}$. Após isso, o método retornará o somatório das menores distâncias.

In [15]:
class KMeans3(KMeans2):
    def score(self, X):
        D = dist_matrix(X, self.W)
        mais_proximos = np.apply_along_axis(np.argmin, 1, D)
        soma_menores = 0
        i = 0
        for j in mais_proximos:
            soma_menores += D[i, j]
            i += 1
        return soma_menores

Exemplo:

In [16]:
# Treinando o modelo com os dados da Questão 6
np.random.seed(7122019)
KM = KMeans3(3, 10)

X_train = 10*np.random.random(30).reshape((10,3))
print(X_train)
print()
KM.fit(X_train)
print(KM.W)
print()

# E testando com a matriz X_test da Questão 7:
np.random.seed(6122019)
X_test = 10*np.random.random(30).reshape((10,3))
print(X_test)
print()
print(KM.predict(X_test))
print(KM.score(X_test))

[[5.98905728 0.85417875 0.71521686]
 [2.70610853 5.5891685  0.94554379]
 [2.4095396  6.33326107 4.33563529]
 [2.16038629 1.07980959 8.18982537]
 [4.12732617 9.50209211 1.79611561]
 [9.12236753 0.93009858 3.38350959]
 [3.28493483 0.45451419 5.70774676]
 [9.07584218 9.67105005 5.54693358]
 [9.94364387 3.37678032 6.67667902]
 [8.70399316 3.27684632 2.6071873 ]]

[[7.71560407 7.51664083 4.67324274]
 [2.72266056 0.76716189 6.94878606]
 [5.78621322 3.39671064 2.39741857]]

[[1.75700393 1.59491685 9.6645511 ]
 [9.255594   7.87658576 9.70541694]
 [6.59413172 7.12764595 0.43241671]
 [1.35201539 8.4707409  6.34732478]
 [4.5306859  3.47221748 5.1326979 ]
 [5.90729181 3.4516044  9.48961166]
 [7.31914307 2.9305075  4.32998883]
 [5.20007376 4.43314179 1.7743091 ]
 [1.57399981 0.45514553 4.40701687]
 [7.23453043 8.83256176 9.52587947]]

[1 0 2 0 2 1 2 2 1 0]
38.817392280125695
